In [154]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV,KFold
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import LabelEncoder


In [155]:
df_train = pd.read_csv('../../../datasets/parte2/treino/dataset_prepared.csv', na_filter= False)
df_test = pd.read_csv('../../../datasets/parte2/teste/dataset_prepared.csv')

In [156]:
X = df_train.drop('injection', axis=1)
y = df_train['injection']

In [158]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [159]:
%pip install -upgrade keras


Usage:   
  /home/goncalo/miniconda3/envs/daa310/bin/python -m pip install [options] <requirement specifier> [package-index-options] ...
  /home/goncalo/miniconda3/envs/daa310/bin/python -m pip install [options] -r <requirements file> [package-index-options] ...
  /home/goncalo/miniconda3/envs/daa310/bin/python -m pip install [options] [-e] <vcs project url> ...
  /home/goncalo/miniconda3/envs/daa310/bin/python -m pip install [options] [-e] <local project path> ...
  /home/goncalo/miniconda3/envs/daa310/bin/python -m pip install [options] <archive url/path> ...

no such option: -u
Note: you may need to restart the kernel to use updated packages.


In [160]:
 # %pip install scikeras[tensorflow]

In [161]:
def build_model(activation = 'relu',learning_rate =0.01):
    model = Sequential()
    model.add(Dense(16,input_dim=11, activation = activation ))
    model.add(Dense(8,activation=activation))
    model.add(Dense(1,activation=activation)) # output 
    #Compile the model
    model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = tf.optimizers.Adam(learning_rate),
    metrics = ['accuracy','recall'])
    return model 

In [162]:
model = build_model()
model.summary()


Model: "sequential_127"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_381 (Dense)           (None, 16)                192       
                                                                 
 dense_382 (Dense)           (None, 8)                 136       
                                                                 
 dense_383 (Dense)           (None, 1)                 9         
                                                                 
Total params: 337 (1.32 KB)
Trainable params: 337 (1.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [163]:
optimizer = ['Adam', 'SGD','RMSProp']
param_grid = dict(optimizer=optimizer)

In [164]:
kf = KFold(n_splits=2, shuffle=True, random_state=2023)

In [165]:
model = KerasClassifier(build_fn=build_model, epochs=1, batch_size=32, verbose=0)


In [166]:
grid_search = GridSearchCV(estimator = model, param_grid= param_grid, cv = kf, refit='True',verbose = 1 )

In [167]:
print(X)
print(y)

        temp  temp_min  temp_max  pressure  humidity  wind_speed  clouds_all  \
0      13.97     11.45     14.04      1027        81        1.96          87   
1      13.48     13.43     13.90      1028        82        1.83          91   
2      12.93     12.72     13.43      1027        84        1.96          93   
3      12.61     10.34     12.93      1027        85        1.85          95   
4      12.61      9.79     12.93      1027        86        1.83          93   
...      ...       ...       ...       ...       ...         ...         ...   
11011  15.70     15.57     15.72      1018        86        5.56         100   
11012  15.55     12.72     15.70      1018        87        4.47         100   
11013  13.45     12.23     13.99      1019        93        3.29         100   
11014  12.93     12.23     13.43      1019        94        1.49         100   
11015  12.93     12.23     13.43      1019        95        1.93         100   

       normal  economic_schedule  self-

In [168]:
grid_search.fit(X,y)

ValueError: Found input variables with inconsistent numbers of samples: [11016, 11017]

In [ ]:
best_mlp_model = grid_search.best_estimator_
print(best_mlp_model)

In [ ]:
best_mlp_model.fit(X,y,epochs = 20, validation_data = (X,y), verbose = 1)

In [ ]:
predictions = best_mlp_model.predict(df_test)

In [ ]:
df_predicoes = pd.DataFrame({'Result': predictions})
df_predicoes['RowId'] = range(1, len(predictions) + 1)
df_predicoes = df_predicoes[['RowId', 'Result']]

df_predicoes.to_csv('../../../datasets/parte2/teste/neuralNetwork.csv', index=False)